# Deploy with Kubernetes

For more detailed explanations, see [README](README.md)

## 1. Let's get started

A Kubernetes multi-node cluster is required to deploy the application.

Check if a cluster, called "smarthome", already exists:

In [ ]:
export CLUSTER=smarthome

# list the clusters
kind get clusters

If it doesn't exist, then create it with Kind by using the manifest ```paas/00-kind_create_cluster.yml```:

In [ ]:
kind create cluster --config paas/00-kind_create_cluster.yml --name $CLUSTER

Configure the K8s connection to the cluster "smarthome":

In [ ]:
export KUBECONFIG="$(kind get kubeconfig-path --name=$CLUSTER)"

Test the connection:

In [ ]:
kubectl cluster-info

The application is delivered via Docker images that are stored into a private repository on DockerHub. So, perform the login to DockerHub via Docker login utility:

In [ ]:
### already logged in
#docker login --username=$USER

Setup a Kubernetes Secret object to allow the pulling of images from the private repository:

In [ ]:
kubectl create secret generic regcred \
    --from-file=.dockerconfigjson=$HOME/.docker/config.json \
    --type=kubernetes.io/dockerconfigjson

## 2. Create a Persistent Volume

Use the manifest ```paas/01-k8s_create_volume.yml``` that describes 3 Kubernetes resources:
- **StorageClass** defines policies for *binding* and *provisioning* the volume
- **PersistentVolume** defines the *node affinity* and the *access mode* to the volume
- **PersistentVolumeClaim** used to specify that a node uses the volume

Manually choose a worker node that hasn't been scheduled yet:

In [ ]:
kubectl get nodes

In [ ]:
WORKER="${CLUSTER}-worker"

Create a mount point inside the chosen node:

In [ ]:
docker exec -it $WORKER sh -c "mkdir /mnt/volume"

Check if the folder has been created

In [ ]:
docker exec -it $WORKER sh -c "ls /mnt"
unset WORKER

Modify the manifest ```paas/01-k8s_create_volume.yml``` to specify the mount point and the correct node reference:

```bash 
vi paas/01-k8s_create_volume.yml
```

Create the persistent volume and its claim with k8s:

In [ ]:
kubectl create -f paas/01-k8s_create_volume.yml

Check the new resources:

In [ ]:
# list the storage classes
kubectl get sc

In [ ]:
# list the persistent volumes
kubectl get pv

In [ ]:
# list the claims
kubectl get pvc

## 3. Deploy Datastore

Use the manifest ```paas/02-k8s_deploy_datastore.yml``` that describes 2 Kubernetes resources:

- **StatefulSet** defines one standalone stateful replica (1 Pod)
- **Service** defines a headless service (required by StatefulSet)

Deploy Datastore with K8s:

In [ ]:
kubectl create -f paas/02-k8s_deploy_datastore.yml

Check the new resources:

In [ ]:
# list the services
kubectl get svc

In [ ]:
# list the stateful sets
kubectl get statefulset

In [ ]:
# list the pods with label app=datastore
kubectl get pods -l app=datastore

Check if the persistent volume has been correctly binded and provisioned:

In [ ]:
# get the node's name
NODE_NAME="$(kubectl get pod -l app=datastore -o json | jq '.items[0].spec.nodeName' | tr -d '"')"

# get the Pod's hostname
POD_NAME="$(kubectl get pod -l app=datastore -o json | jq '.items[0].spec.hostname' | tr -d '"')"

# inspect the mount point on the node container
docker exec -it $NODE_NAME sh -c "ls /mnt/volume"

In [ ]:
# inspect the mounted volume on the Pod
kubectl exec -it $POD_NAME -- sh -c "ls /data/db"
unset POD_NAME

Put a taint "NoSchedule" on the node where Datastore Pod is scheduled:

In [ ]:
kubectl taint node $NODE_NAME node-role.kubernetes.io/master=value:NoSchedule
unset NODE_NAME

## 4. Populate the Datastore

Use the manifest ```paas/03-k8s_create_pod_initdb.yml``` to create a Kubernetes Pod in charge of populating the db directly inside the cluster.

Create the Pod from a Docker image that includes a Python script, for querying the db, and a basic set of information to move inside the db.

Retrieve the connection parameters to Datastore (Pod's hostname and MongoDB's port):

In [ ]:
# get the IP address of the Datastore Pod
export DB_HOST="$(kubectl get pod -l app=datastore -o json | jq '.items[0].status.podIP' | tr -d '"')"

# get the port exposed by the Mongo container
export DB_PORT="$(kubectl get pod -l app=datastore -o json | jq '.items[0].spec.containers[0].ports[0].containerPort' | tr -d '"')"

echo "Pod hostname" $DB_HOST
echo "Exposed port" $DB_PORT

Modify the manifest ```paas/03-k8s_create_pod_initdb.yml``` so that it includes the references to the Datastore Pod:

```bash
vi paas/03-k8s_create_pod_initdb.yml
```

Create the Pod with K8s:

In [ ]:
kubectl create -f paas/03-k8s_create_pod_initdb.yml

Check it the Pod is running:

In [ ]:
kubectl get pod initdb

Manually execute the Python script within the Pod:

In [ ]:
kubectl exec -it initdb -- sh -c "python init.py"

Once finished, delete the Pod:

In [ ]:
kubectl delete pod initdb

Check the running Pods

In [ ]:
kubectl get pods

## 4. Deploy the Corestack

Use the manifest ```paas/04-k8s_deploy_core.yml``` that describes 2 Kubernetes resources:

- **Deployment** setups the ReplicaSet and the service discovery mode (via environment variables)
- **Service** uses NodePort to publish the Core service by allocating a static port

Retrieve the connection parameters to Datastore (pod-hostname:mongo-port)

In [ ]:
echo "Pod hostname" $DB_HOST
echo "Exposed port" $DB_PORT

Modify the manifest ```paas/04-k8s_deploy_core.yml``` to specify the environment variables ```DB_HOST``` and ```DB_PORT```:

```bash
vi paas/04-k8s_deploy_core.yml
```

Deploy the Core stack with K8s:

In [ ]:
kubectl create -f paas/04-k8s_deploy_core.yml

Check the new resources:

In [ ]:
# list the service with label app=core
kubectl get svc -l app=core

In [ ]:
# list the deployment
kubectl get deploy -l app=core

In [ ]:
# list the ReplicaSet
kubectl get rs -l app=core

In [ ]:
# show where the Pods have been scheduled
kubectl get pods -l app=core -o json | jq '.items[] | {name: .metadata.name, node: .spec.nodeName}'

Test the connection to Core service:

In [ ]:
# get the IP address of the master node
export MASTER_NODE="$(docker inspect -f '{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' "$CLUSTER-control-plane")"

# get the port exposed by Core service
export CORE_PORT="$(kubectl get svc -l app=core -o json | jq '.items[0].spec.ports[0].nodePort')"

# perform a "GET /" on Core APIs
curl http://$MASTER_NODE:$CORE_PORT

## 5. Deploy the Dashboard

Use the manifest ```paas/05-k8s_deploy_dashboard.yml``` that describes 2 Kubernetes resources:

- **Deployment** setups the ReplicaSet and the service discovery mode (via environment variables)
- **Service** uses NodePort to publish the Dashboard service by allocating a static port

Retrieve the connection parameters to Corestack (master node IP address and Core service port):

In [ ]:
echo "Master node IP" $MASTER_NODE
echo "Corestack port" $CORE_PORT

Modify the manifest ```paas/05-k8s_deploy_dashboard.yml``` to specify the environment variables ```CORESTACK_HOST``` and ```CORESTACK_PORT```:

```bash
vi paas/05-k8s_deploy_dashboard.yml
```

Deploy the Dashboard with K8s:

In [ ]:
kubectl create -f paas/05-k8s_deploy_dashboard.yml

Check the new resources:

In [ ]:
# list the service with label app=dashboard
kubectl get svc -l app=dashboard

In [ ]:
# list the deployment
kubectl get deploy -l app=dashboard

In [ ]:
# list the ReplicaSet
kubectl get rs -l app=dashboard

In [ ]:
# show where the Pods have been scheduled
kubectl get pods -l app=dashboard -o json | jq '.items[] | {name: .metadata.name, node: .spec.nodeName}'

Test the connection to Dashboard service:

In [ ]:
# get the port exposed by Dashboard service
export DASHBOARD_PORT="$(kubectl get svc -l app=dashboard -o json | jq '.items[0].spec.ports[0].nodePort')"

# perform a "GET /" on Dashboard
curl http://$MASTER_NODE:$DASHBOARD_PORT

# In Production
## 1. Reaching Core and Dashboard from the outside

The exposed services can be accessible from the outside after the SSH client performes a SSH address binding on the remote PaaS VM.

**Connect to Core** 
- Bind the address
  ```bash
  ssh -NL 8050:$MASTER_NODE:$CORE_PORT $USER@$PAAS_HOST
  ```
- Connect via web browser at ```http://localhost:8050```

**Connect to Dashboard**
- Bind the address 
  ```bash
  ssh -NL 8080:$MASTER_NODE:$DASHBOARD_PORT $USER@$PAAS_HOST
  ```
- Connect via web browser at ```http://localhost:8080```

In [ ]:
echo "Master node IP" $MASTER_NODE
echo "Corestack port" $CORE_PORT
echo "Dashboard port" $DASHBOARD_PORT

## 2. New Deployment and Rollout
### 2.1. Deployment of v2

A new version (v2) for the Corestack is available.

Modify the deployment of the Corestack to include the new version of the Docker image:

In [ ]:
kubectl get deploy -l app=core -o yaml | \
sed 's;mcaliandro/smarthome:corev1;mcaliandro/smarthome:corev2;g' | \
kubectl replace -f -

Show the details about the new deployment:

In [ ]:
kubectl get deploy -l app=core

Check the functionalities of Core via web browser at [http://localhost:8050](http://localhost:8050)

Check the functionalities of Dashboard via web browser at [http://localhost:8080](http://localhost:8080)

### 2.2. Rollback Core Deployment to v1

Now, Dashboard simply doesn't work with the new version of Core, it would be better to rollback it to the previous version.

Get the history of the Core deployments:

In [ ]:
kubectl rollout history deploy core-deploy

Rollback to v1, i.e., undo the last deployment of v2:

In [ ]:
kubectl rollout undo deployment core-deploy

## 3. Load Balancing

The load balancing experiment is done by activating the SHS emulator and the House Owner emulator, that perform a block of HTTP requests every second.

Open a new terminal session and monitor the current workload:

```bash
kubectl get pod -o wide -w
```

### 3.1. Activate the SHS Emulator

The set of smart home devices can be emulated via a Python script that performs some PUT requests to the Core APIs, to provide the updated information about the system they monitor. 

Retrieve the connection parameters to Corestack (master node IP address and Core service port):

In [ ]:
echo "Master node IP" $MASTER_NODE
echo "Corestack port" $CORE_PORT

Create a container (outside the cluster) from a Docker image that includes the mechanism to perform PUT requests every 5 seconds:

In [ ]:
docker run -d \
    -e API_HOST=$MASTER_NODE \
    -e API_PORT=$CORE_PORT \
    --name shsemulator mcaliandro/smarthome:shsemulator

Check if the container "shsemulator" is running:

In [ ]:
docker ps -f name=shsemulator

Check the logs within the "shsemulator" container:

In [ ]:
docker logs shsemulator

### 3.2. Activate the House Owner Emulator

The workload from the client-side (the traffic to the Dashboard) can be emulated via a Python script that performs some GET requests to the Dashboard, to retrieve the updated information about the smart home system the users own.

Create a container (outside the cluster) from a Docker image that includes the mechanism to perform GET requests every second:

In [ ]:
echo "Master node IP" $MASTER_NODE
echo "Dashboard port" $DASHBOARD_PORT

In [ ]:
docker run -d \
    -e WEB_HOST=$MASTER_NODE \
    -e WEB_PORT=$DASHBOARD_PORT \
    --name hoemulator mcaliandro/smarthome:hoemulator

Check if the container "hoemulator" is running:

In [ ]:
docker ps -f name=hoemulator

Check the logs within the "hoemulator" container:

In [ ]:
docker logs hoemulator

### 3.3. Scale the Deployment

Scale up the deployment of Core by increasing the number of replicas to 12:

In [ ]:
kubectl scale deploy core-deploy --replicas=12

Scale up the deployment of Dashboard by increasing the number of replicas to 6:

In [ ]:
kubectl scale deploy dashboard-deploy --replicas=6

Inspect the resources consumed by Core Service

In [ ]:
kubectl describe svc -l app=core

Inspect the resources consumed by Dashboard Service

In [ ]:
kubectl describe svc -l app=dashboard

### 3.4. Downscale the Deployment

Stop the "shsemulator" container

In [ ]:
docker stop shsemulator

Stop the "hoemulator" container

In [ ]:
docker stop hoemulator

Downscale Core replicas back to 3:

In [ ]:
kubectl get deploy core-deploy -o yaml | sed 's/replicas: 12/replicas: 3/g' | kubectl replace -f -

Downscale Dashboard replicas back to 3:

In [ ]:
kubectl get deploy dashboard-deploy -o yaml | sed 's/replicas: 6/replicas: 3/g' | kubectl replace -f -

Delete the "shsemulator" container

In [ ]:
docker rm shsemulator

Delete the "hoemulator" container

In [ ]:
docker rm hoemulator

In [ ]:
### THE END ###

# Delete the cluster

In [ ]:
kind delete cluster --name=$CLUSTER

In [ ]:
unset DB_HOST; unset DB_PORT
unset MASTER_NODE; unset CORE_PORT; unset DASHBOARD_PORT
unset CLUSTER; unset KUBECONFIG